In [11]:
from confluent_kafka.admin import AdminClient, NewTopic


admin_client = AdminClient({
    "bootstrap.servers": "localhost:9092"
})

admin_client.list_topics().topics

{'book_schemas': TopicMetadata(book_schemas, 1 partitions),
 'user-tracker': TopicMetadata(user-tracker, 1 partitions),
 '__consumer_offsets': TopicMetadata(__consumer_offsets, 50 partitions)}

In [2]:
topic_list = []
topic_list.append(NewTopic("user-tracker", 1, 1))
admin_client.create_topics(topic_list)

{'user-tracker': <Future at 0x2746b17a340 state=running>}

In [3]:
from confluent_kafka import Producer

producer = Producer({'bootstrap.servers':'localhost:9092'})

In [4]:
def receipt(err,msg):
    if err is not None:
        print('Error: {}'.format(err))
    else:
        message = 'Produced message on topic {} with value of {}\n'.format(msg.topic(), msg.value().decode('utf-8'))
        print(message)

In [5]:
import random,json,time

for i in range(10):
        data={
           'user_id': random.randint(20000, 100000),
           'platform': random.choice(['Mobile', 'Laptop', 'Tablet']),
           }
        msg = json.dumps(data)
        producer.poll(0)
        producer.produce('user-tracker', msg.encode('utf-8'),callback=receipt)
        producer.flush()
        time.sleep(3)

Produced message on topic user-tracker with value of {"user_id": 89159, "platform": "Mobile"}

Produced message on topic user-tracker with value of {"user_id": 88245, "platform": "Tablet"}

Produced message on topic user-tracker with value of {"user_id": 22660, "platform": "Mobile"}

Produced message on topic user-tracker with value of {"user_id": 77976, "platform": "Tablet"}

Produced message on topic user-tracker with value of {"user_id": 54340, "platform": "Tablet"}



KeyboardInterrupt: 

In [4]:
from confluent_kafka import Consumer
import json

consumer = Consumer({'bootstrap.servers':'localhost:9092','group.id':'a', 'auto.offset.reset':'earliest'})
consumer.subscribe(['book_schemas'])

while True:
        msg = consumer.poll(5) #timeout
        if msg is None:
            continue
        if msg.error():
            print('Error: {}'.format(msg.error()))
            continue
        data=msg.value().decode('utf-8')
        result=json.loads(data)
        print(process_book_schema_dict_list(result))
        time.sleep(5)
consumer.close()

KeyboardInterrupt: 